##Parallel Predictions

The two sets of dummy data are showing survey results from 2 campaign strategies. Comparing to the baseline from current strategies<br>
Set A wins more democratic to switch party by 2%, but loses 8% more latino and asian voters*. <br>
Set B is more preferrable by 5% of African Americans*, but 1% of current republicans will not vote for it. <br>

Which strategy should Republicans choose? Which location or racial group should we target with limited resource?

*Data adjustement: 
Baseline: DEM=RAND()*0.003; REP= RAND()*0.003+0.997
Set A: DEM=RAND()*0.003*1.02; REP= RAND()*0.003+0.997; applying a factor of 0.92 for Asian, Latino and Mixed racial groups. <br>
Set B: DEM=RAND()*0.003; REP= (RAND()*0.003+0.997)*0.99; applying a factor of 1.05 for the African American group.

In [0]:
## Initialize
import numpy as np
import pandas as pd
import time
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
## Load data
filename='gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Clean datasets/Denver_Voters_Master_Compact_2019.csv'
filename_A='gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Clean datasets/Dummy Data/Dummy A.csv'
filename_B='gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Clean datasets/Dummy Data/Dummy B.csv'
df = pd.read_csv(filename,dtype={'VoterID':str,'Tract-CensusBlock':str})
df_A = pd.read_csv(filename_A)
df_B = pd.read_csv(filename_B)
### EDA with dummy data to spot the data architecture
print(df.head(),df_A.shape,df_A.head(),df_B.head())

                  Address  ...  Tract-CensusBlock
0          161 S GROVE ST  ...    1080310009032.0
1           44 N PEARL ST  ...    1080310028034.0
2  8101 E MISSISSIPPI AVE  ...    1080310070883.0
3         7101 W YALE AVE  ...    1080310119031.0
4         7101 W YALE AVE  ...    1080310119031.0

[5 rows x 14 columns] (909, 9)    Age Gender  ... Poss2OrMoreRaces  PossHispanicOrLatinoOrigin
0   18      F  ...           0.0023                      0.0003
1   19      F  ...           0.0012                      0.0015
2   20      F  ...           0.0016                      0.0004
3   21      F  ...           0.0020                      0.0011
4   22      F  ...           0.0021                      0.0001

[5 rows x 9 columns]    Age Gender  ... Poss2OrMoreRaces  PossHispanicOrLatinoOrigin
0   18      F  ...           0.0022                      0.0029
1   19      F  ...           0.0026                      0.0002
2   20      F  ...           0.0003                      0.0028
3   21

In [0]:
## Calculate possibility to switch party with dummy datasets.
df_new=pd.merge(df,df_A,how='left',left_on=['Age','Gender','Party'], right_on=['Age','Gender','OriginalParty'])
df['PossA']= df['PercentWhiteAlone']*df_new['PossWhiteAlone']+df['PercentBlackOrAfricanAmericanAlone']*df_new['PossBlackOrAfricanAmericanAlone']+df['PercentAsianAndNativeHawaiianAndOtherPacificIslanderAlone']*df_new['PossAsianAndNativeHawaiianAndOtherPacificIslanderAlone']+df['PercentAmericanIndianAndAlaskaNativeAlone']*df_new['PossAmericanIndianAndAlaskaNativeAlone']+df['Percent2OrMoreRaces']*df_new['Poss2OrMoreRaces']+df['PercentHispanicOrLatinoOrigin']*df_new['PossHispanicOrLatinoOrigin']
df_new=pd.merge(df,df_B,how='left',left_on=['Age','Gender','Party'], right_on=['Age','Gender','OriginalParty'])
df['PossB']= df['PercentWhiteAlone']*df_new['PossWhiteAlone']+df['PercentBlackOrAfricanAmericanAlone']*df_new['PossBlackOrAfricanAmericanAlone']+df['PercentAsianAndNativeHawaiianAndOtherPacificIslanderAlone']*df_new['PossAsianAndNativeHawaiianAndOtherPacificIslanderAlone']+df['PercentAmericanIndianAndAlaskaNativeAlone']*df_new['PossAmericanIndianAndAlaskaNativeAlone']+df['Percent2OrMoreRaces']*df_new['Poss2OrMoreRaces']+df['PercentHispanicOrLatinoOrigin']*df_new['PossHispanicOrLatinoOrigin']
print('Prediction:','\n','Result for Strategy A - ',round(df['PossA'].sum()/100),'\n','Result for Strategy B - ',round(df['PossB'].sum()/100))


Prediction: 
 Result for Strategy A -  52973.0 
 Result for Strategy B -  53063.0


It seems Strategy B is a little bit better than A.

In [0]:
## For Strategy B, which locations or which racial groups is more important?
LocationPoss=df[].groupby('Tract-CensusBlock').PossB.agg(np.mean)
LocationPoss.sort_values(ascending=False).head(10)

Tract-CensusBlock
1080310120012.0    40.944586
1080310120103.0    35.734482
1080310032031.0    33.327075
1080310068091.0    32.989889
1080310120102.0    32.256721
1080310040022.0    30.617563
1080310120101.0    29.704050
1080310055033.0    29.577435
1080310039013.0    29.542976
1080310040024.0    29.039773
Name: PossB, dtype: float64